Deploying Google Kubernetes Cluster

In [ ]:
gcloud beta container --project "fluted-lambda-274409" clusters create "tap-gke-cluster" --zone "us-central1-c" --no-enable-basic-auth --cluster-version "1.21.6-gke.1500" --release-channel "regular" --machine-type "e2-standard-4" --image-type "COS_CONTAINERD" --disk-type "pd-standard" --disk-size "100" --metadata disable-legacy-endpoints=true --scopes "https://www.googleapis.com/auth/devstorage.read_only","https://www.googleapis.com/auth/logging.write","https://www.googleapis.com/auth/monitoring","https://www.googleapis.com/auth/servicecontrol","https://www.googleapis.com/auth/service.management.readonly","https://www.googleapis.com/auth/trace.append" --max-pods-per-node "110" --num-nodes "3" --logging=SYSTEM,WORKLOAD --monitoring=SYSTEM --enable-ip-alias --network "projects/fluted-lambda-274409/global/networks/default" --subnetwork "projects/fluted-lambda-274409/regions/us-central1/subnetworks/default" --no-enable-intra-node-visibility --default-max-pods-per-node "110" --no-enable-master-authorized-networks --addons HorizontalPodAutoscaling,HttpLoadBalancing,GcePersistentDiskCsiDriver --enable-autoupgrade --enable-autorepair --max-surge-upgrade 1 --max-unavailable-upgrade 0 --enable-shielded-nodes --node-locations "us-central1-c"

Set up Kubernetes cluster context

In [ ]:
gcloud container clusters get-credentials tap-gke-cluster --zone us-central1-c --project fluted-lambda-274409

In [ ]:
kubectl config current-context

Download and install Cluster Essentials for VMware Tanzu

In [ ]:
cd /tmp/tanzu-cluster-essentials

In [ ]:
atom install.sh

Copy & Paste the following at the top in install.sh and update the file with actual username and password.

In [ ]:
./install.sh

In [ ]:
kubectl get ns

In [ ]:
cd ~/tanzu
pivnet login --api-token='e938c1500b6f486a8d3f7b9a6bc79672-r'
pivnet download-product-files --product-slug='tanzu-application-platform' --release-version='1.0.1-build.4' --product-file-id=1114446

In [ ]:
tar -xvf tanzu-framework-darwin-amd64.tar

In [ ]:
export TANZU_CLI_NO_INIT=true
tanzu plugin install --local cli all

In [ ]:
tanzu plugin list

Installing Profiles

In [ ]:
cd ~/projects/tap-bounty/install/scripts

In [ ]:
chmod 755 setUp.sh

Edit setUp.sh and update values forI INSTALL_REGISTRY_HOSTNAME, INSTALL_REGISTRY_USERNAME, INSTALL_REGISTRY_PASSWORD, CERTS_DIR, SA_PWD_FILE

In [ ]:
./setUp.sh 

In [ ]:
cd ~/projects/tap-bounty

In [ ]:
tanzu package install tap -p tap.tanzu.vmware.com -v 1.0.0 --values-file tap-values.yml -n tap-install

In [ ]:
tanzu package installed get tap -n tap-install

In [ ]:
tanzu package installed list -A

In [ ]:
kubectl get svc envoy -n tanzu-system-ingress

In [ ]:
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction start --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction add 35.226.28.109 --name="tap-gui.tap.tanzu4u.net." --ttl="300" --type="A" --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction execute --zone="tanzu4u"

In [ ]:
cd tap-ingress

In [ ]:
./configure-ingress.sh values-ingress.yaml

In [ ]:
nslookup tap-gui.tap.tanzu4u.net

In [ ]:
open -a "Google Chrome" https://tap-gui.tap.tanzu4u.net

In [ ]:
tanzu apps workload create tanzu-java-web-app \
--git-repo https://github.com/sreeramsunkara/tanzu-java-web-app \
--git-branch main \
--type web \
--label app.kubernetes.io/part-of=tanzu-java-web-app \
--namespace dev \
--yes

In [ ]:
tanzu apps workload get tanzu-java-web-app -n dev

In [ ]:
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction start --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction add 35.226.28.109 --name="*.apps.tap.tanzu4u.net." --ttl="300" --type="A" --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction execute --zone="tanzu4u"

Postgres Operator installation

In [ ]:
cd ~/projects/tap-bounty/postgres

In [ ]:
chmod 755 prereq.sh

Edit prereq.sh and update username and password for Tanzu Network Registry.

In [ ]:
./prereq.sh

In [ ]:
cat postgres-operator/values.yaml

kubectl create ns postgres-operator

kubectl create secret docker-registry regsecret \
    --docker-server=https://registry.pivotal.io/ \
    --docker-username='sunkarasr@vmware.com' \
    --docker-password='Sreekar)9' \
    --namespace=postgres-operator

helm install my-postgres-operator postgres-operator/ \
  --namespace=postgres-operator \
  --wait    

kubectl get serviceaccount -n postgres-operator

kubectl get all --selector app=postgres-operator -n postgres-operator

kubectl logs -l app=postgres-operator -n postgres-operator

kubectl api-resources --api-group=sql.tanzu.vmware.com -n postgres-operator

In [ ]:
./install-operator.sh

In [ ]:
helm list -n postgres-operator

In [ ]:
kubectl get storageclasses

kubectl create secret docker-registry regsecret \
    --docker-server=https://registry.pivotal.io/ \
    --docker-username='sunkarasr@vmware.com' \
    --docker-password='Sreekar)9' \
    --namespace=dev

kubectl apply -f postgres.yaml

kubectl get postgres/postgres-db -n dev

kubectl apply -f postgres-service-binding-rbac.yaml

In [ ]:
./install-instance.sh

In [ ]:
kubectl get postgres/postgres-db -n dev

Checkout the project https://github.com/sreeramsunkara/spring-petclinic-tap and cd into the folder.
Ensure spring.datasource.initialization-mode=always to create the schema and data application.properties and application-postgres.properties
For subsequent runs set spring.datasource.initialization-mode=never

In [ ]:
cd ~/projects/tap/spring-petclinic-tap

In [ ]:
tanzu apps workload apply -f kubernetes/tap/workload.yaml -y

In [ ]:
tanzu apps workload get spring-petclinic -n dev